In [53]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [54]:
# Get or create SparkContext
sc = SparkContext.getOrCreate()

In [55]:
# Initialize SparkSession
spark = SparkSession(sc)

In [58]:
matrix_a=[((1,1),5),((1,1),0),((2,1),0),((1,2),0)]
matrix_b=[((1,1),5),((1,1),0),((2,1),0),((1,2),0)]

In [59]:
matrix_a

[((1, 1), 5), ((1, 1), 0), ((2, 1), 0), ((1, 2), 0)]

In [60]:
rdd_a=sc.parallelize(matrix_a)
rdd_b=sc.parallelize(matrix_b)
rdd_a

ParallelCollectionRDD[10] at readRDDFromFile at PythonRDD.scala:289

In [47]:
mapped_a=rdd_a.flatMap(lambda x: [((x[0][0],i),('A',x[0][1],x[1])) for i in range(2)])
mapped_b=rdd_b.flatMap(lambda x: [((i,x[0][1]),('B',x[0][0],x[1])) for i in range(2)])
mapped_b

PythonRDD[2] at RDD at PythonRDD.scala:53

In [62]:
# Reduce step
combined_rdd = mapped_a.union(mapped_b).groupByKey()
combined_rdd

PythonRDD[22] at RDD at PythonRDD.scala:53

In [63]:
def multiply_matrices(x):
 result = []
 for val in list(x[1]):
  if val[0] == 'A':
    for val_b in list(x[1]):
      if val_b[0] == 'B':
        result.append(val[1] * val_b[1] * val[2] * val_b[2])
 return (x[0], sum(result))

In [64]:
result_rdd = combined_rdd.map(multiply_matrices)

In [65]:
result=result_rdd.collect()
result

[((1, 1), 25),
 ((2, 0), 0),
 ((0, 2), 0),
 ((0, 1), 0),
 ((1, 2), 0),
 ((1, 0), 0),
 ((2, 1), 0)]